# 학생 이동 및 교환 비용 행렬 계산

In [282]:
import pandas as pd
df = pd.read_excel("최종_그룹_배정_결과.xlsx", sheet_name="신규반번호순")
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_우울_T점수,정서적 민감성_우울_수준,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,45,보통,48,보통,32,낮음,62,높음,1,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,2
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,3
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,4
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,5


In [390]:
# 초기 그룹 컬럼 추가하기 (랜덤 숫자 기반)
import numpy as np
df['초기그룹'] = np.random.randint(0, 11, size=len(df))
df['상담필요'] = np.random.randint(0, 2, size=len(df))
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호,초기그룹,상담필요
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,48,보통,32,낮음,62,높음,1,1,2,0
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,2,2,0
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,3,0,1
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,4,8,0
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,5,5,0


In [391]:
def array_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    이산형 변수의 ideal 대비 actual 차이(diff)와 방향(sign)을 계산하여
    3차원 numpy 배열로 반환하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    selected_discrete_variable : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    diff_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      diff_matrix[g, v, c] = 그룹 g, 변수 v, 카테고리 c 의 diff 값
      diff = ideal_count - actual_count

    sign_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      sign_matrix[g, v, c] = diff의 부호 (+1, 0, -1)

    Notes
    -----
    해당 함수는 기존 딕셔너리 구조를 행렬 구조로 바꿔 compute_discrete_cost 계산을 벡터화하기 위해 수정함
    모든 변수중 가장 많은 카테고리 수에 맞춰 0으로 패딩하여 shape이 동일하도록 구성함
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np
    # 확인해보니 반환값이 그룹 번호 순으로 안나와서 정렬 추가
    groups = sorted(list(group_freq.keys()))
    n_groups = len(groups)
    vars_list = selected_discrete_variable
    n_vars = len(vars_list)
    var_to_idx = {var: idx for idx, var in enumerate(vars_list)} # 예시 {성별: 0, 상담필요: 1}
    category_to_idx = {}
    max_categories = 0
    for var in vars_list:
        categories = set(ideal_freq['population'][var].keys()) # 예시 var 성별이면 {1,2}, 상담필요면 {0,1}
        categories = sorted(list(categories))
        category_to_idx[var] = {cat: idx for idx, cat in enumerate(categories)} # 예시 {성별: {1:0, 2:1}, 상담필요: {0:0, 1:1}}
        if len(categories) > max_categories:
            max_categories = len(categories) # 가장 많은 카테고리 수 갱신
    # 행렬 초기화
    diff_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    sign_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    # 행렬 채우기
    try:
        for g_idx, g in enumerate(groups):
            for var in vars_list:
                v_idx = var_to_idx[var]
                cats = category_to_idx[var]
                for cat, c_idx in cats.items():
                    ideal_count = ideal_freq['population'][var].get(cat, 0)
                    acutual_count = group_freq[g][var].get(cat, 0)
                    diff = ideal_count - acutual_count
                    diff_matrix[g_idx, v_idx, c_idx] = diff
                    sign_matrix[g_idx, v_idx, c_idx] = np.sign(diff)
    except Exception as e:
        print("Error during array_diff_and_sign execution:", str(e))
        raise e
    return diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups

In [285]:
# ideal_freq, group_freq, selected_discrete_variable
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
ideal_freq = compute_ideal_discrete_freq(df, ['성별', '상담필요'])
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
print("Ideal Frequency:", ideal_freq)
print("Group Frequency:", group_freq)

Ideal Frequency: {'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {1: 13.363636363636363, 0: 13.272727272727273}}}
Group Frequency: {np.int32(0): {'성별': {1: 14, 2: 5}, '상담필요': {1: 12, 0: 7}}, np.int32(6): {'성별': {1: 19, 2: 13}, '상담필요': {1: 19, 0: 13}}, np.int32(4): {'성별': {1: 19, 2: 18}, '상담필요': {1: 20, 0: 17}}, np.int32(10): {'성별': {1: 16, 2: 14}, '상담필요': {1: 19, 0: 11}}, np.int32(8): {'성별': {1: 13, 2: 12}, '상담필요': {1: 14, 0: 11}}, np.int32(9): {'성별': {2: 18, 1: 9}, '상담필요': {1: 15, 0: 12}}, np.int32(5): {'성별': {2: 16, 1: 12}, '상담필요': {0: 17, 1: 11}}, np.int32(7): {'성별': {1: 12, 2: 11}, '상담필요': {0: 16, 1: 7}}, np.int32(2): {'성별': {2: 15, 1: 15}, '상담필요': {0: 18, 1: 12}}, np.int32(3): {'성별': {2: 9, 1: 8}, '상담필요': {0: 10, 1: 7}}, np.int32(1): {'성별': {2: 15, 1: 10}, '상담필요': {0: 14, 1: 11}}}


In [286]:
diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups = array_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
print(diff_matrix) # 그룹별 변수별 카테고리별 diff 값
print(sign_matrix) # 그룹별 변수별 카테고리별 sign 값
print(var_to_idx) # 변수명과 인덱스 매핑
print(category_to_idx) # 변수별 카테고리명과 인덱스 매핑
print(groups) # 그룹 번호 리스트

[[[-0.63636364  8.27272727]
  [ 6.27272727  1.36363636]]

 [[ 3.36363636 -1.72727273]
  [-0.72727273  2.36363636]]

 [[-1.63636364 -1.72727273]
  [-4.72727273  1.36363636]]

 [[ 5.36363636  4.27272727]
  [ 3.27272727  6.36363636]]

 [[-5.63636364 -4.72727273]
  [-3.72727273 -6.63636364]]

 [[ 1.36363636 -2.72727273]
  [-3.72727273  2.36363636]]

 [[-5.63636364  0.27272727]
  [ 0.27272727 -5.63636364]]

 [[ 1.36363636  2.27272727]
  [-2.72727273  6.36363636]]

 [[ 0.36363636  1.27272727]
  [ 2.27272727 -0.63636364]]

 [[ 4.36363636 -4.72727273]
  [ 1.27272727 -1.63636364]]

 [[-2.63636364 -0.72727273]
  [ 2.27272727 -5.63636364]]]
[[[-1.  1.]
  [ 1.  1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[-1. -1.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[-1.  1.]
  [ 1. -1.]]

 [[ 1.  1.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1. -1.]]

 [[ 1. -1.]
  [ 1. -1.]]

 [[-1. -1.]
  [ 1. -1.]]]
{'성별': 0, '상담필요': 1}
{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}
[np.int32(0)

In [287]:
# 기존 딕셔너리로 계산하는 함수 가지고 와서 array_diff_and_sign와 동일한지 확인
import pprint
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
pprint.pprint(group_freq)
print(ideal_freq)
def compute_group_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    각 그룹별 이산형 변수의 차이(diff)와 방향(sign)을 계산하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    discrete_vars : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    group_diff_cost : dict
        각 그룹의 변수별 diff와 sign을 담은 딕셔너리
        구조 예시:
        {
            0: {
                '성별_명렬표_diff': {1: 7.5, 2: 9.5},
                '성별_명렬표_sign': {1: +1.0, 2: +1.0},
                '상담 필요_diff': {0: 15.0, 1: 2.0},
                '상담 필요_sign': {0: +1.0, 1: +1.0}
            },
            1: { ... },
            ...
        }

    Notes
    -----
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np

    group_diff_cost = {}
    try:
        for g in group_freq.keys():
            group_diff_cost[g] = {}

            for var in selected_discrete_variable:
                # diff, sign 각각을 별도 dict로 저장
                diff_dict = {}
                sign_dict = {}

                # 각 범주(key)별로 ideal 대비 actual 차이 계산
                for key, ideal_count in ideal_freq['population'][var].items():
                    actual_count = group_freq[g][var].get(key, 0)
                    diff = ideal_count - actual_count
                    diff_dict[key] = diff
                    sign_dict[key] = np.sign(diff) # 이상치보다 작으면 +1, 크면 -1, 같으면 0

                # 변수별 결과를 group_diff_cost에 저장
                group_diff_cost[g][f'{var}_diff'] = diff_dict
                group_diff_cost[g][f'{var}_sign'] = sign_dict
    except Exception as e:
        print("Error during compute_group_diff_and_sign execution:", str(e))
        raise e
    return group_diff_cost
group_diff_cost = compute_group_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
import pprint
pprint.pprint(group_diff_cost)
# 출력값 확인해보니 diff_matrix와 group_diff_cost와 값은 동일하나 그룹 순서가 안맞아서 array_diff_and_sign에 그룹 정렬 추가함
# 행렬의 변수 순서가 다르긴 하지만 이건 var_to_idx로 매핑 가능함

{np.int32(0): {'상담필요': {0: 7, 1: 12}, '성별': {1: 14, 2: 5}},
 np.int32(1): {'상담필요': {0: 14, 1: 11}, '성별': {1: 10, 2: 15}},
 np.int32(2): {'상담필요': {0: 18, 1: 12}, '성별': {1: 15, 2: 15}},
 np.int32(3): {'상담필요': {0: 10, 1: 7}, '성별': {1: 8, 2: 9}},
 np.int32(4): {'상담필요': {0: 17, 1: 20}, '성별': {1: 19, 2: 18}},
 np.int32(5): {'상담필요': {0: 17, 1: 11}, '성별': {1: 12, 2: 16}},
 np.int32(6): {'상담필요': {0: 13, 1: 19}, '성별': {1: 19, 2: 13}},
 np.int32(7): {'상담필요': {0: 16, 1: 7}, '성별': {1: 12, 2: 11}},
 np.int32(8): {'상담필요': {0: 11, 1: 14}, '성별': {1: 13, 2: 12}},
 np.int32(9): {'상담필요': {0: 12, 1: 15}, '성별': {1: 9, 2: 18}},
 np.int32(10): {'상담필요': {0: 11, 1: 19}, '성별': {1: 16, 2: 14}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {1: 13.363636363636363, 0: 13.272727272727273}}}
{np.int32(0): {'상담필요_diff': {0: 6.272727272727273, 1: 1.3636363636363633},
               '상담필요_sign': {0: np.float64(1.0), 1: np.float64(1.0)},
               '성별_diff': {1: -0.6363636363636367, 2

In [288]:
# 출발 그룹 설정이 제대로 되는지 확인
from cost_group_move_v3 import compute_group_total_cost
pprint.pprint(group_freq)
print(ideal_freq)
group_total_cost, _ = compute_group_total_cost(ideal_freq, group_freq, ['성별', '상담필요'])
pprint.pprint(group_total_cost)
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

{np.int32(0): {'상담필요': {0: 7, 1: 12}, '성별': {1: 14, 2: 5}},
 np.int32(1): {'상담필요': {0: 14, 1: 11}, '성별': {1: 10, 2: 15}},
 np.int32(2): {'상담필요': {0: 18, 1: 12}, '성별': {1: 15, 2: 15}},
 np.int32(3): {'상담필요': {0: 10, 1: 7}, '성별': {1: 8, 2: 9}},
 np.int32(4): {'상담필요': {0: 17, 1: 20}, '성별': {1: 19, 2: 18}},
 np.int32(5): {'상담필요': {0: 17, 1: 11}, '성별': {1: 12, 2: 16}},
 np.int32(6): {'상담필요': {0: 13, 1: 19}, '성별': {1: 19, 2: 13}},
 np.int32(7): {'상담필요': {0: 16, 1: 7}, '성별': {1: 12, 2: 11}},
 np.int32(8): {'상담필요': {0: 11, 1: 14}, '성별': {1: 13, 2: 12}},
 np.int32(9): {'상담필요': {0: 12, 1: 15}, '성별': {1: 9, 2: 18}},
 np.int32(10): {'상담필요': {0: 11, 1: 19}, '성별': {1: 16, 2: 14}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {1: 13.363636363636363, 0: 13.272727272727273}}}
{np.int32(0): np.float64(109.23966942148762),
 np.int32(1): np.float64(13.388429752066116),
 np.int32(2): np.float64(-26.148760330578504),
 np.int32(3): np.float64(98.23140495867769),
 np.int32(4):

In [289]:
# 출발 학생 목록 가지고오기
source_group_df = df[df['초기그룹']==source_group_idx]
print(source_group_df['상담필요'].value_counts())
print(source_group_df['성별'].value_counts())

상담필요
1    20
0    17
Name: count, dtype: int64
성별
1    19
2    18
Name: count, dtype: int64


In [290]:
# 타깃그룹 설정
groups = df['초기그룹'].unique()
target_group_list = [g for g in groups if g != source_group_idx]
print("Target Group List (excluding source group):", target_group_list)

# 출발그룹 설정
groups = df['초기그룹'].unique()
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

Target Group List (excluding source group): [np.int32(0), np.int32(6), np.int32(10), np.int32(8), np.int32(9), np.int32(5), np.int32(7), np.int32(2), np.int32(3), np.int32(1)]
Source Group Index with Minimum Total Cost: 4


In [291]:
var_to_idx

{'성별': 0, '상담필요': 1}

In [292]:
category_to_idx

{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}

In [293]:
diff_matrix
# 해당 배열은 아래와 같음
# G, V, C 순서로 되어있음
# G : 그룹 번호
# V : 변수 번호 (0: 성별, 1: 상담필요)
# C : 카테고리 번호 (성별: 0-남,1-여 / 상담필요: 0-필요없음,1-필요함)

array([[[-0.63636364,  8.27272727],
        [ 6.27272727,  1.36363636]],

       [[ 3.36363636, -1.72727273],
        [-0.72727273,  2.36363636]],

       [[-1.63636364, -1.72727273],
        [-4.72727273,  1.36363636]],

       [[ 5.36363636,  4.27272727],
        [ 3.27272727,  6.36363636]],

       [[-5.63636364, -4.72727273],
        [-3.72727273, -6.63636364]],

       [[ 1.36363636, -2.72727273],
        [-3.72727273,  2.36363636]],

       [[-5.63636364,  0.27272727],
        [ 0.27272727, -5.63636364]],

       [[ 1.36363636,  2.27272727],
        [-2.72727273,  6.36363636]],

       [[ 0.36363636,  1.27272727],
        [ 2.27272727, -0.63636364]],

       [[ 4.36363636, -4.72727273],
        [ 1.27272727, -1.63636364]],

       [[-2.63636364, -0.72727273],
        [ 2.27272727, -5.63636364]]])

In [294]:
# 배열의 차원 예시는 아래와 같다
# G : 그룹 번호
diff_matrix[source_group_idx] # 출발 그룹의 변수별 카테고리별 diff 값

array([[-5.63636364, -4.72727273],
       [-3.72727273, -6.63636364]])

In [295]:
# V : 변수 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값
s_row = source_group_df.iloc[0]
print(s_row[['성별', '상담필요']])
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    print(f"Variable: {var}, Index: {v_idx}")
    print(diff_matrix[source_group_idx, v_idx]) # 출발 그룹의 첫번째 변수(성별)의 카테고리별 diff 값

성별      2
상담필요    1
Name: 2, dtype: object
Variable: 성별, Index: 0
[-5.63636364 -4.72727273]
Variable: 상담필요, Index: 1
[-3.72727273 -6.63636364]


In [296]:
# C : 카테고리 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값 중
# 해당 학생이 속한 카테고리의 diff 값
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    cat_value = s_row[var]
    c_idx = category_to_idx[var][cat_value]
    print(f"Variable: {var}, Category Value: {cat_value}, Category Index: {c_idx}")
    print(diff_matrix[source_group_idx, v_idx, c_idx]) # 출발 그룹의 첫번째 변수(성별)의 해당 카테고리의 diff 값

Variable: 성별, Category Value: 2, Category Index: 1
-4.727272727272727
Variable: 상담필요, Category Value: 1, Category Index: 1
-6.636363636363637


In [297]:
# 이동 전 모든 그룹의 diff_matrix 확인
for group_idx in range(diff_matrix.shape[0]):
    print(f"Group Index: {group_idx}")
    print(diff_matrix[group_idx])

Group Index: 0
[[-0.63636364  8.27272727]
 [ 6.27272727  1.36363636]]
Group Index: 1
[[ 3.36363636 -1.72727273]
 [-0.72727273  2.36363636]]
Group Index: 2
[[-1.63636364 -1.72727273]
 [-4.72727273  1.36363636]]
Group Index: 3
[[5.36363636 4.27272727]
 [3.27272727 6.36363636]]
Group Index: 4
[[-5.63636364 -4.72727273]
 [-3.72727273 -6.63636364]]
Group Index: 5
[[ 1.36363636 -2.72727273]
 [-3.72727273  2.36363636]]
Group Index: 6
[[-5.63636364  0.27272727]
 [ 0.27272727 -5.63636364]]
Group Index: 7
[[ 1.36363636  2.27272727]
 [-2.72727273  6.36363636]]
Group Index: 8
[[ 0.36363636  1.27272727]
 [ 2.27272727 -0.63636364]]
Group Index: 9
[[ 4.36363636 -4.72727273]
 [ 1.27272727 -1.63636364]]
Group Index: 10
[[-2.63636364 -0.72727273]
 [ 2.27272727 -5.63636364]]


In [298]:
# 출발 그룹에서 모든 학생들이 도착 그룹으로 이동했을때 diff_matrix 계산
'''
어떻게 생각해야할까?
출발 그룹 중 하나의 학생이
도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?
그면 차원이 어떻게 될까?
    출발 그룹 학생 수 만큼 차원이 늘어나고
    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?
    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?
    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데
'''
# 우선 차원 틀 먼저 만들어보자
# N : 출발 그룹 학생 수
# G : 도착 그룹 수 (출발 그룹 제외해야하지만 계산을 위해 모든 그룹 포함)
# V : 변수 수
# C : 가장 많은 카테고리 수

'\n어떻게 생각해야할까?\n출발 그룹 중 하나의 학생이\n도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?\n그면 차원이 어떻게 될까?\n    출발 그룹 학생 수 만큼 차원이 늘어나고\n    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?\n    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?\n    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데\n'

In [299]:
# 이동 후 diff_matrix 배열 반영
N = source_group_df.shape[0]
G = len(target_group_list) # 타깃 그룹만 -> 출발 그룹은 따로 배열 계산하는것으로 진행 (출발그룹과 도착그룹 모두 같은 배열에 두니까 복잡함...)
V = diff_matrix.shape[1]
C = diff_matrix.shape[2]
print(f"N (출발 그룹 학생 수): {N}, G (전체 그룹 수): {G}, V (변수 수): {V}, C (최대 카테고리 수): {C}")

N (출발 그룹 학생 수): 37, G (전체 그룹 수): 10, V (변수 수): 2, C (최대 카테고리 수): 2


In [300]:
# 출발 그룹 df 인덱스와 행렬 인덱스 매핑
studentidx_to_arrayidx = {}
for idx, student_idx in enumerate(source_group_df.index):
    studentidx_to_arrayidx[student_idx] = idx
print("Source Group DataFrame Index to Array Index Mapping:", studentidx_to_arrayidx)
# 출발 그룹 df 인덱스와 행렬 인덱스 역매핑
arrayidx_to_studentidx = {}
for student_idx, array_idx in studentidx_to_arrayidx.items():
    arrayidx_to_studentidx[array_idx] = student_idx
print("Array Index to Source Group DataFrame Index Mapping:", arrayidx_to_studentidx)

Source Group DataFrame Index to Array Index Mapping: {2: 0, 21: 1, 33: 2, 35: 3, 43: 4, 55: 5, 70: 6, 72: 7, 82: 8, 86: 9, 93: 10, 94: 11, 106: 12, 121: 13, 123: 14, 142: 15, 155: 16, 163: 17, 170: 18, 181: 19, 184: 20, 188: 21, 192: 22, 194: 23, 220: 24, 225: 25, 227: 26, 229: 27, 231: 28, 234: 29, 248: 30, 253: 31, 256: 32, 259: 33, 265: 34, 268: 35, 272: 36}
Array Index to Source Group DataFrame Index Mapping: {0: 2, 1: 21, 2: 33, 3: 35, 4: 43, 5: 55, 6: 70, 7: 72, 8: 82, 9: 86, 10: 93, 11: 94, 12: 106, 13: 121, 14: 123, 15: 142, 16: 155, 17: 163, 18: 170, 19: 181, 20: 184, 21: 188, 22: 192, 23: 194, 24: 220, 25: 225, 26: 227, 27: 229, 28: 231, 29: 234, 30: 248, 31: 253, 32: 256, 33: 259, 34: 265, 35: 268, 36: 272}


In [301]:
# G가 결국 어떠한 값이 정확한 도착 그룹 번호인지 명확하지 않아서
# 맵핑 데이터가 필요함
targetgroup_to_arrayidx = {}
for idx, g in enumerate(target_group_list):
    targetgroup_to_arrayidx[g] = idx
print(targetgroup_to_arrayidx)
# key값과 value값이 바뀐것도 필요한가?
arrayidx_to_targetgroup = {}
for arrayidx, target_group in zip(range(G), target_group_list):
    arrayidx_to_targetgroup[arrayidx] = target_group
print(arrayidx_to_targetgroup)

{np.int32(0): 0, np.int32(6): 1, np.int32(10): 2, np.int32(8): 3, np.int32(9): 4, np.int32(5): 5, np.int32(7): 6, np.int32(2): 7, np.int32(3): 8, np.int32(1): 9}
{0: np.int32(0), 1: np.int32(6), 2: np.int32(10), 3: np.int32(8), 4: np.int32(9), 5: np.int32(5), 6: np.int32(7), 7: np.int32(2), 8: np.int32(3), 9: np.int32(1)}


In [302]:
# 확인용
print(sorted(target_group_list))
print(s_row['초기그룹'])

[np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10)]
4


In [303]:
# 출발그룹 학생들이 도착그룹으로 이동했을 때의 diff 변화량 계산
after_target_diff_delta_matrix = np.zeros((N, G, V, C), dtype=float)  # N x G x V x C 형태의 배열 초기화

for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(G): # 도착 그룹 인덱스
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_target_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = -1.0
# 확인용
print(after_target_diff_delta_matrix.shape)
print(len(source_group_df))

(37, 10, 2, 2)
37


In [304]:
after_target_diff_delta_matrix[:, 0, :, :] # 각각의 학생이 첫번째 도착 그룹으로 이동했을 때의 diff 변화량

array([[[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1., 

In [305]:
# 각각의 학생이 출발 그룹에서 도착그룹으로 이동했을 때 출발 그룹의 diff 변화량
after_source_diff_delta_matrix = np.zeros((N, 1, V, C), dtype=float)  # N x 1 x V x C 형태의 배열 초기화 -> 출발 그룹 하나만 고려
for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(1): # 출발 그룹 하나만 고려
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_source_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = 1.0

In [306]:
print(after_source_diff_delta_matrix.shape)

(37, 1, 2, 2)


In [307]:
after_source_diff_delta_matrix # 각각의 학생이 출발 그룹에서 도착그룹으로 이동했을 때 출발 그룹의 diff 변화량

array([[[[0., 1.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0

In [308]:
# 현재 모든 그룹의 diff_matrix 배열
# 이걸로 출발 그룹의 diff 변화량을 합산해 출발 그룹의 최종 diff_matrix 계산해야함
# 또한 도착 그룹의 diff 변화량 계산에도 사용해야함

diff_matrix.shape # (전체 그룹 수, 변수 수, 최대 카테고리 수)
# 출발 학생 수 고려해서 차원을 확정해야함

# 현재 모든 그룹의 diff_matrix 배열을 출발 학생 수에 맞게 확장
before_diff_matrix = diff_matrix[None, :, :] * np.ones((N, 1, 1, 1))  # N x G x V x C 형태로 확장
print(before_diff_matrix.shape)
# 근데 도착그룹 수만큼 차원을 줄여야함 (위와 같이 하면 출발그룹도 포함되어서 복잡해짐)
before_target_diff_matrix = before_diff_matrix[:, list(targetgroup_to_arrayidx.values()), :, :] # 도착 그룹의 diff_matrix만 남기기
# 확인용
print(before_target_diff_matrix.shape)

# 출발 그룹의 diff_matrix 계산 -> 출발 그룹 1개만 남기기
before_source_diff_matrix = before_diff_matrix[:, source_group_idx:source_group_idx+1, :, :] # 출발 그룹의 diff_matrix만 남기기 -> 1로 넣으니 차원이 생략되서 1:2로 슬라이싱
print(before_source_diff_matrix.shape)

(37, 11, 2, 2)
(37, 10, 2, 2)
(37, 1, 2, 2)


In [309]:
# 이동 후 출발 그룹 diff_matrix 계산
move_source_diff_matrix = before_source_diff_matrix + after_source_diff_delta_matrix
print(move_source_diff_matrix.shape)
print(move_source_diff_matrix)

(37, 1, 2, 2)
[[[[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-2.72727273 -6.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364

In [310]:
# 이동 후 도착 그룹의 diff_matrix 계산
move_target_diff_matrix = before_target_diff_matrix + after_target_diff_delta_matrix
print(move_target_diff_matrix.shape)
print(move_target_diff_matrix)

(37, 10, 2, 2)
[[[[-0.63636364  7.27272727]
   [ 6.27272727  0.36363636]]

  [[ 3.36363636 -2.72727273]
   [-0.72727273  1.36363636]]

  [[-1.63636364 -2.72727273]
   [-4.72727273  0.36363636]]

  [[ 5.36363636  3.27272727]
   [ 3.27272727  5.36363636]]

  [[-5.63636364 -5.72727273]
   [-3.72727273 -7.63636364]]

  [[ 1.36363636 -3.72727273]
   [-3.72727273  1.36363636]]

  [[-5.63636364 -0.72727273]
   [ 0.27272727 -6.63636364]]

  [[ 1.36363636  1.27272727]
   [-2.72727273  5.36363636]]

  [[ 0.36363636  0.27272727]
   [ 2.27272727 -1.63636364]]

  [[ 4.36363636 -5.72727273]
   [ 1.27272727 -2.63636364]]]


 [[[-1.63636364  8.27272727]
   [ 6.27272727  0.36363636]]

  [[ 2.36363636 -1.72727273]
   [-0.72727273  1.36363636]]

  [[-2.63636364 -1.72727273]
   [-4.72727273  0.36363636]]

  [[ 4.36363636  4.27272727]
   [ 3.27272727  5.36363636]]

  [[-6.63636364 -4.72727273]
   [-3.72727273 -7.63636364]]

  [[ 0.36363636 -2.72727273]
   [-3.72727273  1.36363636]]

  [[-6.63636364  0.2727

In [311]:
# move_source_diff_matrix와 move_target_diff_matrix를 이용해
# 이동 후 최종 diff_matrix 계산
'''
move_source_diff_matrix, move_target_diff_matrix 차원이 달라서 문제가 발생함
move_source_diff_matrix: N x 1 x V x C -> 출발그룹만 존재
move_target_diff_matrix: N x G x V x C -> 도착그룹들
move_target_diff_matrix에서 각 학생이 도착그룹으로 이동 시 각 그룹별로 총 diff 값을 계산해서 제일 작읍 그룹하나만 산출해
차원을 줄이면서 move_source_diff_matrix와 차원을 통일시키고 싶은데, 이렇게 되면 도착그룹 배열 인덱스 정보가 사라지면서 문제가 발생함

- 해결책
1. 학생 마다 도착 그룹별 총 비용을 계산해서
2. 학생 마다 최적 도착그룹 index를 별도 배열에 저장
'''

'\nmove_source_diff_matrix, move_target_diff_matrix 차원이 달라서 문제가 발생함\nmove_source_diff_matrix: N x 1 x V x C -> 출발그룹만 존재\nmove_target_diff_matrix: N x G x V x C -> 도착그룹들\nmove_target_diff_matrix에서 각 학생이 도착그룹으로 이동 시 각 그룹별로 총 diff 값을 계산해서 제일 작읍 그룹하나만 산출해\n차원을 줄이면서 move_source_diff_matrix와 차원을 통일시키고 싶은데, 이렇게 되면 도착그룹 배열 인덱스 정보가 사라지면서 문제가 발생함\n\n- 해결책\n1. 학생 마다 도착 그룹별 총 비용을 계산해서\n2. 학생 마다 최적 도착그룹 index를 별도 배열에 저장\n'

In [312]:
print(move_target_diff_matrix[0, :, :, :]) # 첫번째 학생이 각 도착 그룹으로 이동했을 때의 diff_matrix
print(("\n"))

[[[-0.63636364  7.27272727]
  [ 6.27272727  0.36363636]]

 [[ 3.36363636 -2.72727273]
  [-0.72727273  1.36363636]]

 [[-1.63636364 -2.72727273]
  [-4.72727273  0.36363636]]

 [[ 5.36363636  3.27272727]
  [ 3.27272727  5.36363636]]

 [[-5.63636364 -5.72727273]
  [-3.72727273 -7.63636364]]

 [[ 1.36363636 -3.72727273]
  [-3.72727273  1.36363636]]

 [[-5.63636364 -0.72727273]
  [ 0.27272727 -6.63636364]]

 [[ 1.36363636  1.27272727]
  [-2.72727273  5.36363636]]

 [[ 0.36363636  0.27272727]
  [ 2.27272727 -1.63636364]]

 [[ 4.36363636 -5.72727273]
  [ 1.27272727 -2.63636364]]]




In [313]:
print(move_target_diff_matrix[0, 0, :, :]) # 첫번째 학생이 첫번째 도착 그룹으로 이동했을 때의 diff_matrix
print(("\n"))

[[-0.63636364  7.27272727]
 [ 6.27272727  0.36363636]]




In [314]:
# 각 원소의 제곱하여 총합
test = move_target_diff_matrix ** 2
print(test[0, 0, :, :])
print(np.sum(test[0, 0, :, :]))

[[ 0.40495868 52.89256198]
 [39.34710744  0.1322314 ]]
92.77685950413225


In [315]:
# 0단계
# 이동 후 학생들이 출발 그룹에서 떠났을때 비용 계산
## move_source_diff_matrix: shape (N, 1, V, C)
## 차원 (1, V, C) → (V, C)로 축소
## 제곱 후 (V, C)의 모든 원소를 합산
## 각 학생에 대해 반복 → shape (N,)
source_totaldiff_matrix = np.sum(move_source_diff_matrix[:, 0, :, :] ** 2, axis=(1,2))  # N 형태의 배열
print(source_totaldiff_matrix)
print(source_totaldiff_matrix.shape)

[91.32231405 89.50413223 97.14049587 95.32231405 91.32231405 97.14049587
 95.32231405 89.50413223 89.50413223 91.32231405 95.32231405 89.50413223
 97.14049587 89.50413223 89.50413223 91.32231405 91.32231405 97.14049587
 89.50413223 95.32231405 91.32231405 95.32231405 97.14049587 95.32231405
 89.50413223 95.32231405 95.32231405 97.14049587 91.32231405 95.32231405
 91.32231405 91.32231405 91.32231405 95.32231405 97.14049587 89.50413223
 91.32231405]
(37,)


In [316]:
# 이동 후 학생들이 각 도착 그룹으로 갔을때 총 비용 계산
# 1단계
## move_target_diff_matrix: shape (N, G, V, C)
## 제곱 후 (V, C)의 모든 원소를 합산
## 각 학생과 도착 그룹에 대해 반복 → shape (N, G)

target_totaldiff_matrix = np.zeros((N, G), dtype=float)  # N x G 형태의 배열
for student_arrayidx in range(N):
    for targetgroup_arrayidx in range(G):
        total_diff = np.sum(move_target_diff_matrix[student_arrayidx, targetgroup_arrayidx, :, :] ** 2)
        target_totaldiff_matrix[student_arrayidx, targetgroup_arrayidx] = total_diff
print(target_totaldiff_matrix)
print(target_totaldiff_matrix.shape)

[[ 92.7768595   21.14049587  32.59504132  78.95867769 136.7768595
   31.50413223  76.41322314  39.68595041   8.04958678  60.41322314]
 [110.59504132  10.95867769  32.41322314  76.7768595  138.59504132
   23.32231405  88.23140496  41.50413223   9.8677686   42.23140496]
 [ 82.95867769  27.32231405  44.7768595   85.14049587 130.95867769
   43.68595041  64.59504132  57.8677686    2.23140496  54.59504132]
 [100.7768595   17.14049587  44.59504132  82.95867769 132.7768595
   35.50413223  76.41322314  59.68595041   4.04958678  36.41322314]
 [ 92.7768595   21.14049587  32.59504132  78.95867769 136.7768595
   31.50413223  76.41322314  39.68595041   8.04958678  60.41322314]
 [ 82.95867769  27.32231405  44.7768595   85.14049587 130.95867769
   43.68595041  64.59504132  57.8677686    2.23140496  54.59504132]
 [100.7768595   17.14049587  44.59504132  82.95867769 132.7768595
   35.50413223  76.41322314  59.68595041   4.04958678  36.41322314]
 [110.59504132  10.95867769  32.41322314  76.7768595  138.5

In [317]:
# 2단계
## 각 학생이 각 도착그룹으로 이동했을 때 총 비용을 계산하여 N X G 형태의 배열 생성 완료
## 각 학생이 하나의 행이기 때문에 각 행별로 최솟값을 가지는 열 인덱스와 값을 추출
## (N, ) 형태의 배열
target_totaldiff_matrix_min_indices = np.argmin(target_totaldiff_matrix, axis=1)  # N 형태의 배열
print(target_totaldiff_matrix_min_indices)
target_totaldiff_matrix_min_values = np.min(target_totaldiff_matrix, axis=1)  # N 형태의 배열
print(target_totaldiff_matrix_min_values)
print(target_totaldiff_matrix_min_values.shape)

[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[8.04958678 9.8677686  2.23140496 4.04958678 8.04958678 2.23140496
 4.04958678 9.8677686  9.8677686  8.04958678 4.04958678 9.8677686
 2.23140496 9.8677686  9.8677686  8.04958678 8.04958678 2.23140496
 9.8677686  4.04958678 8.04958678 4.04958678 2.23140496 4.04958678
 9.8677686  4.04958678 4.04958678 2.23140496 8.04958678 4.04958678
 8.04958678 8.04958678 8.04958678 4.04958678 2.23140496 9.8677686
 8.04958678]
(37,)


In [325]:
# 3단계
## 각 학생들이 출발그룹에서 떠났을 때 총 diff 값과
## 각 학생들이 각 도착그룹에서 이동했을 때 총 diff 값 중 최소값인 최적 이동의 총 diff 값을 더해서
## 최종 총 diff 배열 완성
after_cost = source_totaldiff_matrix + target_totaldiff_matrix_min_values
print(after_cost)

[99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083 99.37190083
 99.37190083]


In [334]:
# 4단계
## 이동 전 출발 그룹의 총 diff 값과
## 이동 후 도착 그룹의 총 diff 값의 합산
## 이건 각 학생 차원을 고려할 필요가 없지 않나? -> 필요없음 새로운 배열로 계산해야함

# 현재 모든 그룹의 diff_matrix 배열
diff_matrix.shape # (전체 그룹 수, 변수 수, 최대 카테고리 수)
move_before_source_diff_matrix = diff_matrix[source_group_idx:source_group_idx+1, :, :] # 출발 그룹 비용
move_before_target_diff_matrix = diff_matrix[list(targetgroup_to_arrayidx.values()), :, :] # 도착 그룹 비용
print(move_before_source_diff_matrix.shape)
print(move_before_target_diff_matrix.shape)

# move_before_source_diff_matrix이 (1, 2, 2) 형태라서 move_before_target_diff_matrix 형태와 동일하게 맞추고 계산
move_before_source_diff_matrix = move_before_source_diff_matrix * np.ones((G, 1, 1))  # G x V x C 형태로 확장
# 총 diff 값 계산 ()
before_source_totaldiff = np.sum(move_before_source_diff_matrix ** 2, axis=(1,2)) # 그룹별로 합산
before_target_totaldiff = np.sum(move_before_target_diff_matrix ** 2, axis=(1,2))
print(before_source_totaldiff)
print(before_target_totaldiff)
before_cost = before_source_totaldiff + before_target_totaldiff
print(before_cost)
print(before_cost.shape)

(1, 2, 2)
(10, 2, 2)
[112.04958678 112.04958678 112.04958678 112.04958678 112.04958678
 112.04958678 112.04958678 112.04958678 112.04958678 112.04958678]
[110.04958678  20.41322314  29.8677686   98.23140496 112.04958678
  28.7768595   63.68595041  54.95867769   7.32231405  45.68595041]
[222.09917355 132.46280992 141.91735537 210.28099174 224.09917355
 140.82644628 175.73553719 167.00826446 119.37190083 157.73553719]
(10,)


In [335]:
# 5단계
## before_cost - after_cost 계산
## 어떠한 학생이 어떤 도착그룹으로 이동해야 개선폭이 가장 큰지 산출
## before_cost을 학생 수만큼 복제해서 차원 맞추기
print(N)
before_cost = before_cost[None, :] * np.ones((N, 1))  # N, G 형태의 배열
improvement = before_cost - after_cost  # N x G 형태의 배열
print(improvement)

37


ValueError: operands could not be broadcast together with shapes (37,10) (37,) 

In [333]:
print(before_cost[:, None])

[[222.09917355]
 [132.46280992]
 [141.91735537]
 [210.28099174]
 [224.09917355]
 [140.82644628]
 [175.73553719]
 [167.00826446]
 [119.37190083]
 [157.73553719]]


In [ ]:
# ? 단계
## 최종 총 diff 배열에서 최솟값을 가지는 학생 인덱스 추출
## 그 학생이 가는 도착 그룹 인덱스 추출
min_move_student_arrayidx = np.argmin(move_totaldiff_matrix)
print("최소 총 diff 값을 가지는 학생의 배열 인덱스:", min_move_student_arrayidx)
min_move_student_targetgroup_arrayidx = target_totaldiff_matrix_min_indices[min_move_student_arrayidx]
print("최소 총 diff 값을 가지는 학생이 가는 도착 그룹의 배열 인덱스:", min_move_student_targetgroup_arrayidx)

# 이동 전, 이동 후 차원 통일화해서 다시

## 이동 전 비용 계산(before_cost)

### 이동 전 출발그룹 비용 배열(N, T_G, V, C)

In [370]:
# 1단계
## 학생 (출발그룹 안)
## 출발 그룹 -> 2단계에서 동일하게 도착그룹 수만큼 확장
## 변수 수
## 카테고리 수
### 출발 그룹의 이동 전 diff 값 계산
### 모든 학생의 경우가 같음
N = source_group_df.shape[0]
source_group_no = min(group_total_cost, key=group_total_cost.get) # 출발 그룹 번호
G = source_group_no
T_G = len(target_group_list)
V = diff_matrix.shape[1]
C = diff_matrix.shape[2]
print(f"N: {N}명, G: {G}번 그룹, T_G: {T_G}개 도착그룹 수, V: {V}, C: {C}")
before_move_source_diff_matrix = diff_matrix[source_group_no:source_group_no+1, :, :] * np.ones((N, 1, V, C))  # N x G x V x C 형태의 배열 -> 출발그룹 하나만 (확장 전)
print(before_move_source_diff_matrix.shape)
print(before_move_source_diff_matrix)

N: 37명, G: 4번 그룹, T_G: 10개 도착그룹 수, V: 2, C: 2
(37, 1, 2, 2)
[[[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.636

In [371]:
# 2단계
## 출발그룹 하나만되어 있기 때문에 향후 비용 계산을 위해
## 도착그룹 수만큼 차원을 확장해야함
before_move_source_diff_matrix = before_move_source_diff_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태의 배열
print(before_move_source_diff_matrix.shape)
print(before_move_source_diff_matrix)

(37, 10, 2, 2)
[[[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]]


 [[[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[-5.63636364 -4.7272

In [372]:
# 3단계
## 출발그룹 df의 각 학생의 인데스와 배열의 인덱스를 다르기 때문에 매핑 필요
studentidx_to_arrayidx = {}
for idx, student_idx in enumerate(source_group_df.index):
    studentidx_to_arrayidx[student_idx] = idx
## 역매핑
arrayidx_to_studentidx = {}
for student_idx, array_idx in studentidx_to_arrayidx.items():
    arrayidx_to_studentidx[array_idx] = student_idx

### 이동 전 도착그룹 비용배열(N, T_G, V, C)

In [373]:
# 1단계
## 학생 (출발그룹 안)
## 도착 그룹 (출발그룹 제외)
## 변수 수
## 카테고리 수
### 도착 그룹의 이동 전 diff 값 계산
before_move_target_diff_matrix = diff_matrix[list(targetgroup_to_arrayidx.values()), :, :] * np.ones((N, 1, V, C))  # N x T_G x V x C 형태의 배열
print(before_move_target_diff_matrix.shape)
print(before_move_target_diff_matrix)

(37, 10, 2, 2)
[[[[-0.63636364  8.27272727]
   [ 6.27272727  1.36363636]]

  [[ 3.36363636 -1.72727273]
   [-0.72727273  2.36363636]]

  [[-1.63636364 -1.72727273]
   [-4.72727273  1.36363636]]

  [[ 5.36363636  4.27272727]
   [ 3.27272727  6.36363636]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[ 1.36363636 -2.72727273]
   [-3.72727273  2.36363636]]

  [[-5.63636364  0.27272727]
   [ 0.27272727 -5.63636364]]

  [[ 1.36363636  2.27272727]
   [-2.72727273  6.36363636]]

  [[ 0.36363636  1.27272727]
   [ 2.27272727 -0.63636364]]

  [[ 4.36363636 -4.72727273]
   [ 1.27272727 -1.63636364]]]


 [[[-0.63636364  8.27272727]
   [ 6.27272727  1.36363636]]

  [[ 3.36363636 -1.72727273]
   [-0.72727273  2.36363636]]

  [[-1.63636364 -1.72727273]
   [-4.72727273  1.36363636]]

  [[ 5.36363636  4.27272727]
   [ 3.27272727  6.36363636]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[ 1.36363636 -2.72727273]
   [-3.72727273  2.36363636]]

  [[-5.63636364  0.2727

In [374]:
# 2단계
## 이동 전 도착그룹 diff_matrix의 idx에 따라 어떤 도착그룹인지 매핑 필요
targetgroupno_to_arrayidx = {}
for idx, g in enumerate(target_group_list):
    targetgroupno_to_arrayidx[g] = idx
print(targetgroupno_to_arrayidx)
## 역매핑
arrayidx_to_targetgroupno = {}
for target_group_no, array_idx in targetgroupno_to_arrayidx.items():
    arrayidx_to_targetgroupno[array_idx] = target_group_no
print(arrayidx_to_targetgroupno)

{np.int32(0): 0, np.int32(6): 1, np.int32(10): 2, np.int32(8): 3, np.int32(9): 4, np.int32(5): 5, np.int32(7): 6, np.int32(2): 7, np.int32(3): 8, np.int32(1): 9}
{0: np.int32(0), 1: np.int32(6), 2: np.int32(10), 3: np.int32(8), 4: np.int32(9), 5: np.int32(5), 6: np.int32(7), 7: np.int32(2), 8: np.int32(3), 9: np.int32(1)}


### 이동 전 총 diff 비용 계산


In [375]:
before_cost = before_move_source_diff_matrix + before_move_target_diff_matrix
print(before_cost.shape)
print(before_cost)

(37, 10, 2, 2)
[[[[ -6.27272727   3.54545455]
   [  2.54545455  -5.27272727]]

  [[ -2.27272727  -6.45454545]
   [ -4.45454545  -4.27272727]]

  [[ -7.27272727  -6.45454545]
   [ -8.45454545  -5.27272727]]

  [[ -0.27272727  -0.45454545]
   [ -0.45454545  -0.27272727]]

  [[-11.27272727  -9.45454545]
   [ -7.45454545 -13.27272727]]

  [[ -4.27272727  -7.45454545]
   [ -7.45454545  -4.27272727]]

  [[-11.27272727  -4.45454545]
   [ -3.45454545 -12.27272727]]

  [[ -4.27272727  -2.45454545]
   [ -6.45454545  -0.27272727]]

  [[ -5.27272727  -3.45454545]
   [ -1.45454545  -7.27272727]]

  [[ -1.27272727  -9.45454545]
   [ -2.45454545  -8.27272727]]]


 [[[ -6.27272727   3.54545455]
   [  2.54545455  -5.27272727]]

  [[ -2.27272727  -6.45454545]
   [ -4.45454545  -4.27272727]]

  [[ -7.27272727  -6.45454545]
   [ -8.45454545  -5.27272727]]

  [[ -0.27272727  -0.45454545]
   [ -0.45454545  -0.27272727]]

  [[-11.27272727  -9.45454545]
   [ -7.45454545 -13.27272727]]

  [[ -4.27272727  -7.45

In [376]:
before_cost = np.sum(before_cost ** 2, axis=(2,3))  # N x T_G 형태의 배열
print(before_cost.shape)
print(before_cost)

(37, 10)
[[ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 448.19834711
  147.65289256 309.47107438  66.01652893  94.74380165 165.47107438]
 [ 86.19834711  84.92561983 193.83471074   0.5

## 이동 후 비용 계산(after_cost)

### 이동 후 출발그룹 비용 배열

In [377]:
# 1단계
## 각각의 학생이 출발그룹에서 도착그룹으로 이동했을 때 출발그룹의 diff 변화량
after_move_source_diff_delta_matrix = np.zeros((N, 1, V, C), dtype=float)  # N x 1 x V x C 형태의 배열 초기화 -> 출발 그룹 하나만 고려
for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(1): # 출발 그룹 하나만 고려
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_move_source_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = 1.0
print(after_move_source_diff_delta_matrix.shape)
print(after_move_source_diff_delta_matrix)

(37, 1, 2, 2)
[[[[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]]]


In [378]:
# 2단계
## 출발그룹 하나만되어 있기 때문에 향후 비용 계산을 위해
## 도착그룹 수만큼 차원을 확장해야함
after_move_source_diff_delta_matrix = after_move_source_diff_delta_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태의 배열
print(after_move_source_diff_delta_matrix.shape)
print(after_move_source_diff_delta_matrix)

(37, 10, 2, 2)
[[[[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]

  [[0. 1.]
   [1. 0.]]]


 [[[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]

  [[1. 0.]
   [1. 0.]]]


 [[[0. 1.]
   [0.

In [379]:
# 3단계
## 변화량 반영
before_diff_matrix = diff_matrix[None, :, :] * np.ones((N, 1, 1, 1))  # N x G x V x C 형태로 확장
before_source_diff_matrix = before_diff_matrix[:, source_group_idx:source_group_idx+1, :, :] # 출발 그룹의 diff_matrix만 남기기 -> 1로 넣으니 차원이 생략되서 1:2로 슬라이싱
before_source_diff_matrix = before_source_diff_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태로 확장
after_move_source_diff_matrix = before_source_diff_matrix + after_move_source_diff_delta_matrix
print(after_move_source_diff_matrix.shape)
print(after_move_source_diff_matrix)

(37, 10, 2, 2)
[[[[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]

  [[-5.63636364 -3.72727273]
   [-3.72727273 -5.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.72727273]
   [-3.72727273 -5.63636364]]

  [[-4.63636364 -4.7272

### 이동 후 도착그룹 비용 배열

In [380]:
# 1단계
## 출발그룹 학생들이 도착그룹으로 이동했을 때의 diff 변화량 계산
after_move_target_diff_delta_matrix = np.zeros((N, T_G, V, C), dtype=float)  # N x T_G x V x C 형태의 배열 초기화
for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(G): # 도착 그룹 인덱스
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_move_target_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = -1.0
print(after_move_target_diff_delta_matrix.shape)
print(after_move_target_diff_delta_matrix)

(37, 10, 2, 2)
[[[[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[ 0. -1.]
   [-1.  0.]]

  [[ 0. -1.]
   [-1.  0.]]

  [[ 0. -1.]
   [-1.  0.]]

  [[ 0. -1.]
   [-1.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]


 [[[-1.  0.]
   [-1.  0.]]

  [[-1.  0.]
   [-1.  0.]]

  [[-1.  0.]
   [-1.  0.]]

  [[-1.  0.]
   [-1.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]


In [381]:
# 2단계
## 변화량 반영
after_move_target_diff_matrix = before_move_target_diff_matrix + after_move_target_diff_delta_matrix
print(after_move_target_diff_matrix.shape)
print(after_move_target_diff_matrix)

(37, 10, 2, 2)
[[[[-0.63636364  7.27272727]
   [ 6.27272727  0.36363636]]

  [[ 3.36363636 -2.72727273]
   [-0.72727273  1.36363636]]

  [[-1.63636364 -2.72727273]
   [-4.72727273  0.36363636]]

  [[ 5.36363636  3.27272727]
   [ 3.27272727  5.36363636]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[ 1.36363636 -2.72727273]
   [-3.72727273  2.36363636]]

  [[-5.63636364  0.27272727]
   [ 0.27272727 -5.63636364]]

  [[ 1.36363636  2.27272727]
   [-2.72727273  6.36363636]]

  [[ 0.36363636  1.27272727]
   [ 2.27272727 -0.63636364]]

  [[ 4.36363636 -4.72727273]
   [ 1.27272727 -1.63636364]]]


 [[[-1.63636364  8.27272727]
   [ 6.27272727  0.36363636]]

  [[ 2.36363636 -1.72727273]
   [-0.72727273  1.36363636]]

  [[-2.63636364 -1.72727273]
   [-4.72727273  0.36363636]]

  [[ 4.36363636  4.27272727]
   [ 3.27272727  5.36363636]]

  [[-5.63636364 -4.72727273]
   [-3.72727273 -6.63636364]]

  [[ 1.36363636 -2.72727273]
   [-3.72727273  2.36363636]]

  [[-5.63636364  0.2727

### 이동 후 총 diff 비용 계산

In [382]:
after_cost = after_move_source_diff_matrix + after_move_target_diff_matrix
print(after_cost.shape)
print(after_cost)

(37, 10, 2, 2)
[[[[ -6.27272727   3.54545455]
   [  2.54545455  -5.27272727]]

  [[ -2.27272727  -6.45454545]
   [ -4.45454545  -4.27272727]]

  [[ -7.27272727  -6.45454545]
   [ -8.45454545  -5.27272727]]

  [[ -0.27272727  -0.45454545]
   [ -0.45454545  -0.27272727]]

  [[-11.27272727  -8.45454545]
   [ -7.45454545 -12.27272727]]

  [[ -4.27272727  -6.45454545]
   [ -7.45454545  -3.27272727]]

  [[-11.27272727  -3.45454545]
   [ -3.45454545 -11.27272727]]

  [[ -4.27272727  -1.45454545]
   [ -6.45454545   0.72727273]]

  [[ -5.27272727  -2.45454545]
   [ -1.45454545  -6.27272727]]

  [[ -1.27272727  -8.45454545]
   [ -2.45454545  -7.27272727]]]


 [[[ -6.27272727   3.54545455]
   [  2.54545455  -5.27272727]]

  [[ -2.27272727  -6.45454545]
   [ -4.45454545  -4.27272727]]

  [[ -7.27272727  -6.45454545]
   [ -8.45454545  -5.27272727]]

  [[ -0.27272727  -0.45454545]
   [ -0.45454545  -0.27272727]]

  [[-10.27272727  -9.45454545]
   [ -7.45454545 -12.27272727]]

  [[ -3.27272727  -7.45

In [383]:
after_cost = np.sum(after_cost ** 2, axis=(2,3))  # N x T_G 형태의 배열
print(after_cost.shape)
print(after_cost)

(37, 10)
[[ 86.19834711  84.92561983 193.83471074   0.56198347 404.74380165
  126.19834711 278.01652893  62.56198347  75.2892562  132.01652893]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 401.10743802
  132.56198347 264.38016529  58.92561983  71.65289256 148.38016529]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 416.38016529
  119.83471074 295.65289256  50.19834711  86.92561983 143.65289256]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 412.74380165
  126.19834711 282.01652893  46.56198347  83.2892562  160.01652893]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 404.74380165
  126.19834711 278.01652893  62.56198347  75.2892562  132.01652893]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 416.38016529
  119.83471074 295.65289256  50.19834711  86.92561983 143.65289256]
 [ 86.19834711  84.92561983 193.83471074   0.56198347 412.74380165
  126.19834711 282.01652893  46.56198347  83.2892562  160.01652893]
 [ 86.19834711  84.92561983 193.83471074   0.5

## 이동 전후 비용 개선 배열

### 총 diff 변화량 계산

In [384]:
cost_change = before_cost - after_cost
print(cost_change.shape)
print(cost_change)

(37, 10)
[[ 0.          0.          0.          0.         43.45454545 21.45454545
  31.45454545  3.45454545 19.45454545 33.45454545]
 [ 0.          0.          0.          0.         47.09090909 15.09090909
  45.09090909  7.09090909 23.09090909 17.09090909]
 [ 0.          0.          0.          0.         31.81818182 27.81818182
  13.81818182 15.81818182  7.81818182 21.81818182]
 [ 0.          0.          0.          0.         35.45454545 21.45454545
  27.45454545 19.45454545 11.45454545  5.45454545]
 [ 0.          0.          0.          0.         43.45454545 21.45454545
  31.45454545  3.45454545 19.45454545 33.45454545]
 [ 0.          0.          0.          0.         31.81818182 27.81818182
  13.81818182 15.81818182  7.81818182 21.81818182]
 [ 0.          0.          0.          0.         35.45454545 21.45454545
  27.45454545 19.45454545 11.45454545  5.45454545]
 [ 0.          0.          0.          0.         47.09090909 15.09090909
  45.09090909  7.09090909 23.09090909 17.0

### 최적 학생 및 도착그룹

In [385]:
# 1단계
## 각행이 학생이므로 각 행별로 최댓값을 가지는 열 인덱스와 값을 추출
max_cost_change_indices = np.argmax(cost_change, axis=1)  # N 형태의 배열
print(max_cost_change_indices)
max_cost_change_values = np.max(cost_change, axis=1)  # N 형태의 배열
print(max_cost_change_values)
## 모든 행 중 최댓값을 가지는 행 인덱스 추출
max_cost_change_overall_index = np.argmax(max_cost_change_values)
print("최대 비용 변화를 가지는 학생의 배열 인덱스:", max_cost_change_overall_index)
max_cost_change_overall_value = max_cost_change_values[max_cost_change_overall_index]
print("최대 비용 변화 값:", max_cost_change_overall_value)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[43.45454545 47.09090909 31.81818182 35.45454545 43.45454545 31.81818182
 35.45454545 47.09090909 47.09090909 43.45454545 35.45454545 47.09090909
 31.81818182 47.09090909 47.09090909 43.45454545 43.45454545 31.81818182
 47.09090909 35.45454545 43.45454545 35.45454545 31.81818182 35.45454545
 47.09090909 35.45454545 35.45454545 31.81818182 43.45454545 35.45454545
 43.45454545 43.45454545 43.45454545 35.45454545 31.81818182 47.09090909
 43.45454545]
최대 비용 변화를 가지는 학생의 배열 인덱스: 1
최대 비용 변화 값: 47.090909090909065


In [387]:
arrayidx_to_targetgroupno[4]

np.int32(9)

In [389]:
df_9 = df[df['초기그룹']==arrayidx_to_targetgroupno[4]]
print(df_9['상담필요'].value_counts())
print(df_9['성별'].value_counts())

상담필요
1    15
0    12
Name: count, dtype: int64
성별
2    18
1     9
Name: count, dtype: int64


## 함수로 리펙토링

### 현재 그룹 diff 배열 계산

In [397]:
def array_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    이산형 변수의 ideal 대비 actual 차이(diff)와 방향(sign)을 계산하여
    3차원 numpy 배열로 반환하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    selected_discrete_variable : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    diff_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      diff_matrix[g, v, c] = 그룹 g, 변수 v, 카테고리 c 의 diff 값
      diff = ideal_count - actual_count

    sign_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      sign_matrix[g, v, c] = diff의 부호 (+1, 0, -1)

    Notes
    -----
    해당 함수는 기존 딕셔너리 구조를 행렬 구조로 바꿔 compute_discrete_cost 계산을 벡터화하기 위해 수정함
    모든 변수중 가장 많은 카테고리 수에 맞춰 0으로 패딩하여 shape이 동일하도록 구성함
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np
    # 확인해보니 반환값이 그룹 번호 순으로 안나와서 정렬 추가
    groups = sorted(list(group_freq.keys()))
    n_groups = len(groups)
    vars_list = selected_discrete_variable
    n_vars = len(vars_list)
    var_to_idx = {var: idx for idx, var in enumerate(vars_list)} # 예시 {성별: 0, 상담필요: 1}
    category_to_idx = {}
    max_categories = 0
    for var in vars_list:
        categories = set(ideal_freq['population'][var].keys()) # 예시 var 성별이면 {1,2}, 상담필요면 {0,1}
        categories = sorted(list(categories))
        category_to_idx[var] = {cat: idx for idx, cat in enumerate(categories)} # 예시 {성별: {1:0, 2:1}, 상담필요: {0:0, 1:1}}
        if len(categories) > max_categories:
            max_categories = len(categories) # 가장 많은 카테고리 수 갱신
    # 행렬 초기화
    diff_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    sign_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    # 행렬 채우기
    try:
        for g_idx, g in enumerate(groups):
            for var in vars_list:
                v_idx = var_to_idx[var]
                cats = category_to_idx[var]
                for cat, c_idx in cats.items():
                    ideal_count = ideal_freq['population'][var].get(cat, 0)
                    acutual_count = group_freq[g][var].get(cat, 0)
                    diff = ideal_count - acutual_count
                    diff_matrix[g_idx, v_idx, c_idx] = diff
                    sign_matrix[g_idx, v_idx, c_idx] = np.sign(diff)
    except Exception as e:
        print("Error during array_diff_and_sign execution:", str(e))
        raise e
    return diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups

In [398]:
# 이상적 빈도 및 그룹별 빈도 계산
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
ideal_freq = compute_ideal_discrete_freq(df, ['성별', '상담필요'])
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
print("Ideal Frequency:", ideal_freq)
print("Group Frequency:", group_freq)

Ideal Frequency: {'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 14.272727272727273, 1: 12.363636363636363}}}
Group Frequency: {np.int32(2): {'성별': {1: 18, 2: 18}, '상담필요': {0: 21, 1: 15}}, np.int32(0): {'성별': {2: 16, 1: 15}, '상담필요': {1: 17, 0: 14}}, np.int32(8): {'성별': {2: 19, 1: 14}, '상담필요': {0: 17, 1: 16}}, np.int32(5): {'성별': {1: 17, 2: 9}, '상담필요': {1: 17, 0: 9}}, np.int32(10): {'성별': {1: 11, 2: 10}, '상담필요': {0: 15, 1: 6}}, np.int32(1): {'성별': {1: 19, 2: 14}, '상담필요': {0: 18, 1: 15}}, np.int32(7): {'성별': {2: 7, 1: 7}, '상담필요': {1: 7, 0: 7}}, np.int32(4): {'성별': {1: 18, 2: 14}, '상담필요': {0: 20, 1: 12}}, np.int32(9): {'성별': {2: 16, 1: 10}, '상담필요': {1: 14, 0: 12}}, np.int32(3): {'성별': {2: 15, 1: 14}, '상담필요': {0: 19, 1: 10}}, np.int32(6): {'성별': {2: 8, 1: 4}, '상담필요': {1: 7, 0: 5}}}


In [399]:
diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups = array_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
print(diff_matrix) # 그룹별 변수별 카테고리별 diff 값
print(sign_matrix) # 그룹별 변수별 카테고리별 sign 값
print(var_to_idx) # 변수명과 인덱스 매핑
print(category_to_idx) # 변수별 카테고리명과 인덱스 매핑
print(groups) # 그룹 번호 리스트

[[[-1.63636364 -2.72727273]
  [ 0.27272727 -4.63636364]]

 [[-5.63636364 -0.72727273]
  [-3.72727273 -2.63636364]]

 [[-4.63636364 -4.72727273]
  [-6.72727273 -2.63636364]]

 [[-0.63636364 -1.72727273]
  [-4.72727273  2.36363636]]

 [[-4.63636364 -0.72727273]
  [-5.72727273  0.36363636]]

 [[-3.63636364  4.27272727]
  [ 5.27272727 -4.63636364]]

 [[ 9.36363636  5.27272727]
  [ 9.27272727  5.36363636]]

 [[ 6.36363636  6.27272727]
  [ 7.27272727  5.36363636]]

 [[-0.63636364 -5.72727273]
  [-2.72727273 -3.63636364]]

 [[ 3.36363636 -2.72727273]
  [ 2.27272727 -1.63636364]]

 [[ 2.36363636  3.27272727]
  [-0.72727273  6.36363636]]]
[[[-1. -1.]
  [ 1. -1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[-1. -1.]
  [-1.  1.]]

 [[-1. -1.]
  [-1.  1.]]

 [[-1.  1.]
  [ 1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1. -1.]
  [ 1. -1.]]

 [[ 1.  1.]
  [-1.  1.]]]
{'성별': 0, '상담필요': 1}
{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}
[np.int32(0)

### 현재 그룹의 총 비용 산출

In [400]:
# 출발 그룹 설정이 제대로 되는지 확인
from cost_group_move_v3 import compute_group_total_cost
pprint.pprint(group_freq)
print(ideal_freq)
group_total_cost, _ = compute_group_total_cost(ideal_freq, group_freq, ['성별', '상담필요'])
pprint.pprint(group_total_cost)
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

{np.int32(0): {'상담필요': {0: 14, 1: 17}, '성별': {1: 15, 2: 16}},
 np.int32(1): {'상담필요': {0: 18, 1: 15}, '성별': {1: 19, 2: 14}},
 np.int32(2): {'상담필요': {0: 21, 1: 15}, '성별': {1: 18, 2: 18}},
 np.int32(3): {'상담필요': {0: 19, 1: 10}, '성별': {1: 14, 2: 15}},
 np.int32(4): {'상담필요': {0: 20, 1: 12}, '성별': {1: 18, 2: 14}},
 np.int32(5): {'상담필요': {0: 9, 1: 17}, '성별': {1: 17, 2: 9}},
 np.int32(6): {'상담필요': {0: 5, 1: 7}, '성별': {1: 4, 2: 8}},
 np.int32(7): {'상담필요': {0: 7, 1: 7}, '성별': {1: 7, 2: 7}},
 np.int32(8): {'상담필요': {0: 17, 1: 16}, '성별': {1: 14, 2: 19}},
 np.int32(9): {'상담필요': {0: 12, 1: 14}, '성별': {1: 10, 2: 16}},
 np.int32(10): {'상담필요': {0: 15, 1: 6}, '성별': {1: 11, 2: 10}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 14.272727272727273, 1: 12.363636363636363}}}
{np.int32(0): np.float64(-31.537190082644628),
 np.int32(1): np.float64(-53.14049586776859),
 np.int32(2): np.float64(-96.0495867768595),
 np.int32(3): np.float64(-20.148760330578504),
 np.int32(4): np

### 최적 이동 함수

In [ ]:
def array_discrete_cost(source_group_df, group_total_cost, target_group_list, diff_matrix, var_to_idx, category_to_idx):
    #######################################################################################
    # 이동 전 비용 계산
    ## 이동 전 출발그룹 비용 배열
    ## 1단계
    N = source_group_df.shape[0]
    source_group_no = min(group_total_cost, key=group_total_cost.get) # 출발 그룹 번호
    G = source_group_no
    T_G = len(target_group_list)
    V = diff_matrix.shape[1]
    C = diff_matrix.shape[2]
    print(f"N: {N}명, G: {G}번 그룹, T_G: {T_G}개 도착그룹 수, V: {V}, C: {C}")
    before_move_source_diff_matrix = diff_matrix[source_group_no:source_group_no+1, :, :] * np.ones((N, 1, V, C))  # N x G x V x C 형태의 배열 -> 출발그룹 하나만 (확장 전)
    print(before_move_source_diff_matrix.shape)
    print(before_move_source_diff_matrix)
    ## 2단계
    before_move_source_diff_matrix = before_move_source_diff_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태의 배열
    print(before_move_source_diff_matrix.shape)
    print(before_move_source_diff_matrix)
    ## 3단계
    ## 출발그룹 df의 각 학생의 인데스와 배열의 인덱스를 다르기 때문에 매핑 필요
    studentidx_to_arrayidx = {}
    for idx, student_idx in enumerate(source_group_df.index):
        studentidx_to_arrayidx[student_idx] = idx
    ## 역매핑
    arrayidx_to_studentidx = {}
    for student_idx, array_idx in studentidx_to_arrayidx.items():
        arrayidx_to_studentidx[array_idx] = student_idx

    # 이동 전 도착그룹 비용 배열
    ## 1단계
    before_move_target_diff_matrix = diff_matrix[list(targetgroup_to_arrayidx.values()), :, :] * np.ones((N, 1, V, C))  # N x T_G x V x C 형태의 배열
    print(before_move_target_diff_matrix.shape)
    print(before_move_target_diff_matrix)
    ## 2단계
    ## 이동 전 도착그룹 diff_matrix의 idx에 따라 어떤 도착그룹인지 매핑 필요
    targetgroupno_to_arrayidx = {}
    for idx, g in enumerate(target_group_list):
        targetgroupno_to_arrayidx[g] = idx
    print(targetgroupno_to_arrayidx)
    ## 역매핑
    arrayidx_to_targetgroupno = {}
    for target_group_no, array_idx in targetgroupno_to_arrayidx.items():
        arrayidx_to_targetgroupno[array_idx] = target_group_no
    print(arrayidx_to_targetgroupno)

    # 이동 전 총 diff 비용 계산
    before_cost = before_move_source_diff_matrix + before_move_target_diff_matrix
    print(before_cost.shape)
    print(before_cost)
    before_cost = np.sum(before_cost ** 2, axis=(2,3))  # N x T_G 형태의 배열
    print(before_cost.shape)
    print(before_cost)
    #######################################################################################
    #######################################################################################
    # 이동 후 비용 계산
    ## 이동 후 출발그룹 비용 배열
    ### 1단계
    ### 각각의 학생이 출발그룹에서 도착그룹으로 이동했을 때 출발그룹의 diff 변화량
    after_move_source_diff_delta_matrix = np.zeros((N, 1, V, C), dtype=float)  # N x 1 x V x C 형태의 배열 초기화 -> 출발 그룹 하나만 고려
    for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
        student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
        for targetgroup_arrayidx in range(1): # 출발 그룹 하나만 고려
            for var in ['성별', '상담필요']:
                v_idx = var_to_idx[var]
                cat_value = s_row[var]
                c_idx = category_to_idx[var][cat_value]
                # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
                after_move_source_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = 1.0
    print(after_move_source_diff_delta_matrix.shape)
    print(after_move_source_diff_delta_matrix)
    ### 2단계
    ### 출발그룹 하나만되어 있기 때문에 향후 비용 계산을 위해
    ### 도착그룹 수만큼 차원을 확장해야함
    after_move_source_diff_delta_matrix = after_move_source_diff_delta_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태의 배열
    print(after_move_source_diff_delta_matrix.shape)
    print(after_move_source_diff_delta_matrix)
    ### 3단계
    ### 변화량 반영
    before_diff_matrix = diff_matrix[None, :, :] * np.ones((N, 1, 1, 1))  # N x G x V x C 형태로 확장
    before_source_diff_matrix = before_diff_matrix[:, source_group_idx:source_group_idx+1, :, :] # 출발 그룹의 diff_matrix만 남기기 -> 1로 넣으니 차원이 생략되서 1:2로 슬라이싱
    before_source_diff_matrix = before_source_diff_matrix * np.ones((1, T_G, 1, 1))  # N x T_G x V x C 형태로 확장
    after_move_source_diff_matrix = before_source_diff_matrix + after_move_source_diff_delta_matrix
    print(after_move_source_diff_matrix.shape)
    print(after_move_source_diff_matrix)
    
    ## 이동 후 도착그룹 비용 배열
    ### 1단계
    ### 출발그룹 학생들이 도착그룹으로 이동했을 때의 diff 변화량 계산
    after_move_target_diff_delta_matrix = np.zeros((N, T_G, V, C), dtype=float)  # N x T_G x V x C 형태의 배열 초기화
    for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
        student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
        for targetgroup_arrayidx in range(G): # 도착 그룹 인덱스
            for var in ['성별', '상담필요']:
                v_idx = var_to_idx[var]
                cat_value = s_row[var]
                c_idx = category_to_idx[var][cat_value]
                # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
                after_move_target_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = -1.0
    print(after_move_target_diff_delta_matrix.shape)
    print(after_move_target_diff_delta_matrix)
    ### 2단계
    ### 변화량 반영
    after_move_target_diff_matrix = before_move_target_diff_matrix + after_move_target_diff_delta_matrix
    print(after_move_target_diff_matrix.shape)
    print(after_move_target_diff_matrix)

    ## 이동 후 총 diff 비용 계산
    after_cost = after_move_source_diff_matrix + after_move_target_diff_matrix
    print(after_cost.shape)
    print(after_cost)
    after_cost = np.sum(after_cost ** 2, axis=(2,3))  # N x T_G 형태의 배열
    print(after_cost.shape)
    print(after_cost)
    #######################################################################################
    #######################################################################################
    # 이동 전후 비용 개선 배열
    ## 총 diff 변화량 계산
    cost_change = before_cost - after_cost
    print(cost_change.shape)
    print(cost_change)
    ## 개선폭이 감소하는 경우 음수이므로 무한대 비용 처리
    cost_change[cost_change < 0] = -np.inf
    print(cost_change)
    #######################################################################################
    #######################################################################################
    # # 최적 학생 및 도착그룹
    # ## 1단계
    # ### 각행이 학생이므로 각 행별로 최댓값을 가지는 열 인덱스와 값을 추출
    # max_cost_change_targetgroup_indices = np.argmax(cost_change, axis=1)  # N 형태의 배열
    # print(max_cost_change_targetgroup_indices)
    # max_cost_change_targetgroup_values = np.max(cost_change, axis=1)  # N 형태의 배열
    # print(max_cost_change_targetgroup_values)
    # ## 모든 행 중 최댓값을 가지는 행 인덱스 추출
    # max_cost_change_student_index = np.argmax(max_cost_change_targetgroup_values)
    # print("최대 비용 변화를 가지는 학생의 배열 인덱스:", max_cost_change_student_index)
    # max_cost_change_overall_value = max_cost_change_targetgroup_values[max_cost_change_student_index]
    # print("최대 비용 변화 값:", max_cost_change_overall_value)
    return cost_change, arrayidx_to_studentidx, arrayidx_to_targetgroupno

In [405]:
cost_change, arrayidx_to_studentidx, arrayidx_to_targetgroupno = array_discrete_cost(source_group_df, group_total_cost, target_group_list, diff_matrix)

N: 37명, G: 2번 그룹, T_G: 10개 도착그룹 수, V: 2, C: 2
(37, 1, 2, 2)
[[[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.63636364 -4.72727273]
   [-6.72727273 -2.63636364]]]


 [[[-4.636

# 연속형 이동 전후 비용 함수 검증

In [411]:
selected_continuous_variables = ['기초학습역량_논리수학_T점수', '기초학습역량_언어_T점수']
N = len(source_group_df)
M = len(target_group_list)
V = len(selected_continuous_variables)

In [412]:
# 출발 그룹 번호
source_group_idx = int(source_group_df["초기그룹"].iloc[0])
print("Source Group Index:", source_group_idx)

Source Group Index: 4


In [415]:
# 출발그룹 학생별 연속형 값 (N, V)
X = source_group_df[selected_continuous_variables].to_numpy()
print("학생별 연속형 값 X shape:", X.shape)
print(X)

학생별 연속형 값 X shape: (37, 2)
[[86 81]
 [86 81]
 [65 77]
 [65 77]
 [65 77]
 [86 81]
 [31 32]
 [86 81]
 [65 77]
 [65 77]
 [65 77]
 [31 32]
 [31 32]
 [31 32]
 [86 81]
 [86 81]
 [31 32]
 [86 81]
 [65 77]
 [65 77]
 [86 81]
 [65 77]
 [65 77]
 [65 77]
 [86 81]
 [86 81]
 [31 32]
 [65 77]
 [86 81]
 [65 77]
 [65 77]
 [65 77]
 [23 23]
 [86 81]
 [65 77]
 [23 23]
 [65 77]]


In [417]:
# 전체 평균 (V,)
pop_mean = df[selected_continuous_variables].mean().to_numpy()
print("전체 평균 pop_mean shape:", pop_mean.shape)
print(pop_mean)

전체 평균 pop_mean shape: (2,)
[51.36860068 53.3447099 ]


In [ ]:
# -------------------------------------
# 1. 출발 그룹 mean / size
# -------------------------------------
source_df = df[df["초기그룹"] == source_group_idx]
mean_s = source_df[selected_continuous_variables].mean().to_numpy()   # 출발그룹 평균 (V,)
size_s = len(source_df)
print("출발 그룹 mean_s shape:", mean_s.shape)
print(mean_s)
print("출발 그룹 size_s:", size_s)
print(size_s)

출발 그룹 mean_s shape: (2,)
[61.15625 62.59375]
출발 그룹 size_s: 32
32


In [ ]:
# -------------------------------------
# 2. 타깃 그룹 mean / size
# -------------------------------------
mean_t_list = []
size_t_list = []
for g in target_group_list:
    gdf = df[df["초기그룹"] == g] # 타깃 그룹 데이터프레임 분리

    mean_t_list.append(gdf[selected_continuous_variables].mean().to_numpy()) # 타깃 그룹 별 평균 (V,)
    size_t_list.append(len(gdf))                             # 타깃 그룹 별 크기
    print(f"타깃 그룹 {g} mean_t shape:", mean_t_list[-1].shape)
    print(f"타깃 그룹 {g} size_t:", size_t_list[-1])

mean_t = np.vstack(mean_t_list)           # 타깃 그룹 별 평균 행렬 (M, V)
size_t = np.array(size_t_list)            # 타깃 그룹 별 크기 배열 (M,)
print("타깃 그룹 mean_t shape:", mean_t.shape)
print(mean_t)
print("타깃 그룹 size_t shape:", size_t.shape)
print(size_t)

타깃 그룹 0 mean_t shape: (2,)
타깃 그룹 0 size_t: 31
타깃 그룹 6 mean_t shape: (2,)
타깃 그룹 6 size_t: 12
타깃 그룹 10 mean_t shape: (2,)
타깃 그룹 10 size_t: 21
타깃 그룹 8 mean_t shape: (2,)
타깃 그룹 8 size_t: 33
타깃 그룹 9 mean_t shape: (2,)
타깃 그룹 9 size_t: 26
타깃 그룹 5 mean_t shape: (2,)
타깃 그룹 5 size_t: 26
타깃 그룹 7 mean_t shape: (2,)
타깃 그룹 7 size_t: 14
타깃 그룹 2 mean_t shape: (2,)
타깃 그룹 2 size_t: 36
타깃 그룹 3 mean_t shape: (2,)
타깃 그룹 3 size_t: 29
타깃 그룹 1 mean_t shape: (2,)
타깃 그룹 1 size_t: 33
타깃 그룹 mean_t shape: (10, 2)
[[47.25806452 50.80645161]
 [57.83333333 59.58333333]
 [49.66666667 50.28571429]
 [48.33333333 51.        ]
 [52.30769231 55.07692308]
 [54.53846154 56.07692308]
 [45.14285714 48.        ]
 [52.05555556 53.61111111]
 [46.20689655 46.93103448]
 [50.6969697  52.87878788]]
타깃 그룹 size_t shape: (10,)
[31 12 21 33 26 26 14 36 29 33]


In [ ]:
# -------------------------------------
# 3. 이동 전 거리 계산 before_dist
# -------------------------------------
# (1, M, V) 형태 만들기
before = (
    np.abs(mean_s - pop_mean)[None, None, :] +   # 출발그룹 평균 - 전체평균 (1,1,V)
    np.abs(mean_t - pop_mean)[None, :, :]        # 타깃 그룹 별 평균 - 전체평균 (1,M,V)
)
print("이동 전 거리 before shape:", before.shape)
print(before)
# (N, M, V)로 확장
before = np.repeat(before, N, axis=0) # 출발그룹 학생 별로 확장
print("이동 전 거리 before 확장 shape:", before.shape)
print(before)

이동 전 거리 before shape: (1, 10, 2)
[[[13.89818548 11.78729839]
  [16.25238197 15.48766354]
  [11.48958333 12.30803571]
  [12.82291667 11.59375   ]
  [10.72674094 10.98125328]
  [12.95751017 11.98125328]
  [16.01339286 14.59375   ]
  [10.47460419  9.51544132]
  [14.94935345 15.66271552]
  [10.4592803   9.71496212]]]
이동 전 거리 before 확장 shape: (37, 10, 2)
[[[13.89818548 11.78729839]
  [16.25238197 15.48766354]
  [11.48958333 12.30803571]
  [12.82291667 11.59375   ]
  [10.72674094 10.98125328]
  [12.95751017 11.98125328]
  [16.01339286 14.59375   ]
  [10.47460419  9.51544132]
  [14.94935345 15.66271552]
  [10.4592803   9.71496212]]

 [[13.89818548 11.78729839]
  [16.25238197 15.48766354]
  [11.48958333 12.30803571]
  [12.82291667 11.59375   ]
  [10.72674094 10.98125328]
  [12.95751017 11.98125328]
  [16.01339286 14.59375   ]
  [10.47460419  9.51544132]
  [14.94935345 15.66271552]
  [10.4592803   9.71496212]]

 [[13.89818548 11.78729839]
  [16.25238197 15.48766354]
  [11.48958333 12.30803571]


In [426]:
# 4-1) 출발 그룹에서 학생 빠질 때 평균 (N, V)
new_mean_s = (mean_s * size_s - X) / (size_s - 1) # (mean_s * size_s - X)으로 다시 총량을 만들고 size_s - 1로 나눔
print("출발 그룹에서 학생 빠질 때 새로운 평균 new_mean_s shape:", new_mean_s.shape)
print(new_mean_s)

# 4-2) 타깃 그룹에서 학생 추가할 때 평균 (N, M, V)
# mean_t: (M,V) → (1,M,V)  
# size_t: (M,)  → (1,M,1)
# X:      (N,V) → (N,1,V)
new_mean_t = (
    (mean_t * size_t[:, None])[None, :, :] + X[:, None, :]
) / (size_t[:, None] + 1)[None, :, :]
print("타깃 그룹에서 학생 추가할 때 새로운 평균 new_mean_t shape:", new_mean_t.shape)
print(new_mean_t)

출발 그룹에서 학생 빠질 때 새로운 평균 new_mean_s shape: (37, 2)
[[60.35483871 62.        ]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [60.35483871 62.        ]
 [62.12903226 63.58064516]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [62.12903226 63.58064516]
 [62.12903226 63.58064516]
 [62.12903226 63.58064516]
 [60.35483871 62.        ]
 [60.35483871 62.        ]
 [62.12903226 63.58064516]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [60.35483871 62.        ]
 [60.35483871 62.        ]
 [62.12903226 63.58064516]
 [61.03225806 62.12903226]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [61.03225806 62.12903226]
 [62.38709677 63.87096774]
 [60.35483871 62.        ]
 [61.03225806 62.12903226]
 [62.3

In [429]:
# -------------------------------------
# 5. 이동 후 거리 after_dist (N,M,V)
# -------------------------------------
after = (
    np.abs(new_mean_s[:, None, :] - pop_mean) +
    np.abs(new_mean_t - pop_mean)
)
print("이동 후 거리 after shape:", after.shape)
print(after)

이동 후 거리 after shape: (37, 10, 2)
[[[11.88608871 10.25      ]
  [17.61763734 16.54134944]
  [ 9.03665689 10.31818182]
  [10.91366224 10.11764706]
  [11.1731929  11.34761724]
  [13.32134105 12.3105802 ]
  [12.48817204 11.8       ]
  [10.59061032  9.66193156]
  [12.82150538 13.93333333]
  [ 9.35293146  9.01646256]]

 [[11.88608871 10.25      ]
  [17.61763734 16.54134944]
  [ 9.03665689 10.31818182]
  [10.91366224 10.11764706]
  [11.1731929  11.34761724]
  [13.32134105 12.3105802 ]
  [12.48817204 11.8       ]
  [10.59061032  9.66193156]
  [12.82150538 13.93333333]
  [ 9.35293146  9.01646256]]

 [[13.21975806 10.50403226]
  [16.67967208 16.36268939]
  [10.6686217  10.62903226]
  [12.20872865 10.36432638]
  [11.07283448 11.32850135]
  [13.22098263 12.29146431]
  [14.5655914  12.19569892]
  [10.7004621   9.68285571]
  [14.19892473 14.19569892]
  [ 9.91461101  9.02784776]]

 [[13.21975806 10.50403226]
  [16.67967208 16.36268939]
  [10.6686217  10.62903226]
  [12.20872865 10.36432638]
  [11.072

In [431]:
improvement = before - after
print("연속형 변수 개선폭 improvement shape:", improvement.shape)
print(improvement)

연속형 변수 개선폭 improvement shape: (37, 10, 2)
[[[ 2.01209677  1.53729839]
  [-1.36525538 -1.0536859 ]
  [ 2.45292644  1.9898539 ]
  [ 1.90925443  1.47610294]
  [-0.44645196 -0.36636396]
  [-0.36383087 -0.32932692]
  [ 3.52522081  2.79375   ]
  [-0.11600613 -0.14649024]
  [ 2.12784807  1.72938218]
  [ 1.10634884  0.69849956]]

 [[ 2.01209677  1.53729839]
  [-1.36525538 -1.0536859 ]
  [ 2.45292644  1.9898539 ]
  [ 1.90925443  1.47610294]
  [-0.44645196 -0.36636396]
  [-0.36383087 -0.32932692]
  [ 3.52522081  2.79375   ]
  [-0.11600613 -0.14649024]
  [ 2.12784807  1.72938218]
  [ 1.10634884  0.69849956]]

 [[ 0.67842742  1.28326613]
  [-0.42729012 -0.87502585]
  [ 0.82096163  1.67900346]
  [ 0.61418801  1.22942362]
  [-0.34609353 -0.34724807]
  [-0.26347245 -0.31021103]
  [ 1.44780146  2.39805108]
  [-0.22585791 -0.16741439]
  [ 0.75042872  1.46701659]
  [ 0.5446693   0.68711436]]

 [[ 0.67842742  1.28326613]
  [-0.42729012 -0.87502585]
  [ 0.82096163  1.67900346]
  [ 0.61418801  1.22942362]


In [432]:
continuous_cost_matrix = improvement.mean(axis=2)
print("연속형 변수 비용 행렬 continuous_cost_matrix shape:", continuous_cost_matrix.shape)
print(continuous_cost_matrix)

연속형 변수 비용 행렬 continuous_cost_matrix shape: (37, 10)
[[ 1.77469758 -1.20947064  2.22139017  1.69267868 -0.40640796 -0.3465789
   3.15948541 -0.13124818  1.92861513  0.9024242 ]
 [ 1.77469758 -1.20947064  2.22139017  1.69267868 -0.40640796 -0.3465789
   3.15948541 -0.13124818  1.92861513  0.9024242 ]
 [ 0.98084677 -0.65115798  1.24998254  0.92180582 -0.3466708  -0.28684174
   1.92292627 -0.19663615  1.10872265  0.61589183]
 [ 0.98084677 -0.65115798  1.24998254  0.92180582 -0.3466708  -0.28684174
   1.92292627 -0.19663615  1.10872265  0.61589183]
 [ 0.98084677 -0.65115798  1.24998254  0.92180582 -0.3466708  -0.28684174
   1.92292627 -0.19663615  1.10872265  0.61589183]
 [ 1.77469758 -1.20947064  2.22139017  1.69267868 -0.40640796 -0.3465789
   3.15948541 -0.13124818  1.92861513  0.9024242 ]
 [-1.52772177  1.11311001 -1.81966555 -1.51415244 -0.15790139 -0.09807233
  -1.98460061 -0.720945   -1.48213756 -1.57654103]
 [ 1.77469758 -1.20947064  2.22139017  1.69267868 -0.40640796 -0.3465789
   

In [519]:
# 이동전 출발그룹 diff 행령
before_move_source_diff_matrix = np.load('before_move_source_diff_matrix.npy')
print(before_move_source_diff_matrix.shape)
print(before_move_source_diff_matrix)

(27, 10, 2, 2)
[[[[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]]


 [[[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.4545

In [520]:
# 이동 전 도착그룹 diff 행렬
before_move_target_diff_matrix = np.load('before_move_target_diff_matrix.npy')
print(before_move_target_diff_matrix.shape)
print(before_move_target_diff_matrix)

(27, 10, 2, 2)
[[[[ 1.09090909 -1.45454545]
   [-0.63636364  0.27272727]]

  [[-0.90909091  1.54545455]
   [ 1.36363636 -0.72727273]]

  [[-0.90909091  0.54545455]
   [ 1.36363636 -1.72727273]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 0.09090909 -0.45454545]
   [-0.63636364  0.27272727]]

  [[ 0.09090909  0.54545455]
   [ 1.36363636 -0.72727273]]

  [[-0.90909091  0.54545455]
   [-1.63636364  1.27272727]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[-0.90909091  1.54545455]
   [ 2.36363636 -1.72727273]]

  [[-0.90909091  1.54545455]
   [ 1.36363636 -0.72727273]]]


 [[[ 1.09090909 -1.45454545]
   [-0.63636364  0.27272727]]

  [[-0.90909091  1.54545455]
   [ 1.36363636 -0.72727273]]

  [[-0.90909091  0.54545455]
   [ 1.36363636 -1.72727273]]

  [[ 1.09090909 -1.45454545]
   [-1.63636364  1.27272727]]

  [[ 0.09090909 -0.45454545]
   [-0.63636364  0.27272727]]

  [[ 0.09090909  0.54545455]
   [ 1.36363636 -0.72727273]]

  [[-0.90909091  0.5454

In [521]:
# 이동 전 총 diff 행렬
before_cost = np.load('before_cost.npy')
print(before_cost.shape)
print(before_cost)

(27, 10)
[[11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.20661157]
 [11.38842975 13.20661157 13.57024793 15.20661157  8.29752066 10.29752066
  13.02479339 15.20661157 19.38842975 13.2

In [522]:
# 이동 후 출발그룹 diff 변화량
after_move_source_diff_delta_matrix = np.load('after_move_source_diff_delta_matrix.npy')
print(after_move_source_diff_delta_matrix.shape)
print(after_move_source_diff_delta_matrix)

(27, 10, 2, 2)
[[[[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]]


 [[[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]

  [[1. 0.]
   [0. 1.]]]


 [[[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]

  [[0. 1.]
   [0. 1.]]]


 [[[1. 0.]
   [1.

In [523]:
# 이동 후 출발그룹 diff 변화량 반영
after_move_source_diff_matrix = np.load('after_move_source_diff_matrix.npy')
print(after_move_source_diff_matrix.shape)
print(after_move_source_diff_matrix)

(27, 10, 2, 2)
[[[[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]]


 [[[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.45454545]
   [-1.63636364  2.27272727]]

  [[ 2.09090909 -1.4545

In [524]:
# 이동 후 도착그룹 diff 변화량
after_move_target_diff_delta_matrix = np.load('after_move_target_diff_delta_matrix.npy')
print(after_move_target_diff_delta_matrix.shape)
print(after_move_target_diff_delta_matrix)

(27, 10, 2, 2)
[[[[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]]


 [[[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]]


 [[[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]

  [[-1.  0.]
   [ 0. -1.]]]


 [[[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]

  [[ 0. -1.]
   [ 0. -1.]]


In [525]:
# 이동 후 도착그룹 diff 변화량 반영
after_move_target_diff_matrix = np.load('after_move_target_diff_matrix.npy')
print(after_move_target_diff_matrix.shape)
print(after_move_target_diff_matrix)

(27, 10, 2, 2)
[[[[ 0.09090909 -1.45454545]
   [-0.63636364 -0.72727273]]

  [[-1.90909091  1.54545455]
   [ 1.36363636 -1.72727273]]

  [[-1.90909091  0.54545455]
   [ 1.36363636 -2.72727273]]

  [[ 0.09090909 -1.45454545]
   [-1.63636364  0.27272727]]

  [[-0.90909091 -0.45454545]
   [-0.63636364 -0.72727273]]

  [[-0.90909091  0.54545455]
   [ 1.36363636 -1.72727273]]

  [[-1.90909091  0.54545455]
   [-1.63636364  0.27272727]]

  [[ 0.09090909 -1.45454545]
   [-1.63636364  0.27272727]]

  [[-1.90909091  1.54545455]
   [ 2.36363636 -2.72727273]]

  [[-1.90909091  1.54545455]
   [ 1.36363636 -1.72727273]]]


 [[[ 0.09090909 -1.45454545]
   [-0.63636364 -0.72727273]]

  [[-1.90909091  1.54545455]
   [ 1.36363636 -1.72727273]]

  [[-1.90909091  0.54545455]
   [ 1.36363636 -2.72727273]]

  [[ 0.09090909 -1.45454545]
   [-1.63636364  0.27272727]]

  [[-0.90909091 -0.45454545]
   [-0.63636364 -0.72727273]]

  [[-0.90909091  0.54545455]
   [ 1.36363636 -1.72727273]]

  [[-1.90909091  0.5454

In [526]:
# 이동 후 총 diff 행렬
after_cost = np.load('after_cost.npy')
print(after_cost.shape)
print(after_cost)

(27, 10)
[[17.38842975 25.20661157 27.57024793 19.20661157 16.29752066 20.29752066
  21.02479339 19.20661157 33.38842975 25.20661157]
 [17.38842975 25.20661157 27.57024793 19.20661157 16.29752066 20.29752066
  21.02479339 19.20661157 33.38842975 25.20661157]
 [17.38842975 25.20661157 27.57024793 19.20661157 16.29752066 20.29752066
  21.02479339 19.20661157 33.38842975 25.20661157]
 [17.38842975 15.20661157 19.57024793 19.20661157 12.29752066 14.29752066
  13.02479339 19.20661157 23.38842975 15.20661157]
 [13.38842975 15.20661157 15.57024793 19.20661157 12.29752066 10.29752066
  21.02479339 19.20661157 19.38842975 15.20661157]
 [17.38842975 25.20661157 27.57024793 19.20661157 16.29752066 20.29752066
  21.02479339 19.20661157 33.38842975 25.20661157]
 [17.38842975 15.20661157 19.57024793 19.20661157 12.29752066 14.29752066
  13.02479339 19.20661157 23.38842975 15.20661157]
 [17.38842975 25.20661157 27.57024793 19.20661157 16.29752066 20.29752066
  21.02479339 19.20661157 33.38842975 25.2

In [527]:
a = before_move_source_diff_matrix + before_move_target_diff_matrix
a = np.sum(a ** 2, axis=(2,3))
print(a.shape)
print(a)

(27, 10)
[[20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.4

In [528]:
b = after_move_target_diff_matrix + after_move_source_diff_matrix
b = np.sum(b ** 2, axis=(2,3))
print(b.shape)
print(b)

(27, 10)
[[20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.41322314]
 [20.7768595   0.41322314  1.14049587 30.41322314 12.59504132  2.59504132
  18.04958678 30.41322314  0.7768595   0.4

In [529]:
# 최종 비용 행렬 계산
cost_change = before_cost - after_cost
print(cost_change.shape)
print(cost_change)

(27, 10)
[[-6.00000000e+00 -1.20000000e+01 -1.40000000e+01 -4.00000000e+00
  -8.00000000e+00 -1.00000000e+01 -8.00000000e+00 -4.00000000e+00
  -1.40000000e+01 -1.20000000e+01]
 [-6.00000000e+00 -1.20000000e+01 -1.40000000e+01 -4.00000000e+00
  -8.00000000e+00 -1.00000000e+01 -8.00000000e+00 -4.00000000e+00
  -1.40000000e+01 -1.20000000e+01]
 [-6.00000000e+00 -1.20000000e+01 -1.40000000e+01 -4.00000000e+00
  -8.00000000e+00 -1.00000000e+01 -8.00000000e+00 -4.00000000e+00
  -1.40000000e+01 -1.20000000e+01]
 [-6.00000000e+00 -2.00000000e+00 -6.00000000e+00 -4.00000000e+00
  -4.00000000e+00 -4.00000000e+00  0.00000000e+00 -4.00000000e+00
  -4.00000000e+00 -2.00000000e+00]
 [-2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -4.00000000e+00
  -4.00000000e+00 -1.77635684e-15 -8.00000000e+00 -4.00000000e+00
  -3.55271368e-15 -2.00000000e+00]
 [-6.00000000e+00 -1.20000000e+01 -1.40000000e+01 -4.00000000e+00
  -8.00000000e+00 -1.00000000e+01 -8.00000000e+00 -4.00000000e+00
  -1.40000000e+01 -1.200

In [530]:
cost_change = a-

SyntaxError: invalid syntax (3611655582.py, line 1)